In [200]:
import pandas as pd
import numpy as np
import json

In [201]:
def clean_yelp(df1,df2):
    final = pd.concat([df1,df2])
    final = final.drop_duplicates(subset = "id",keep = "first").reset_index(drop = True)
    my_dict = dict(final["categories"])
    my_list = []
    for entry in range(len(my_dict)):
        x = my_dict[entry].strip("[")
        x = x.strip(']')
        x = x.replace("'",'"')
        x = x.replace("}, {",",")
        x = json.loads(x)
        my_list.append(x['alias'])
    final["Restaurant Category"] = pd.Series(my_list)
    pickup_list = []
    delivery_list = []
    reserve_list = []
    for entry in range(len(final['transactions'])):
        y = final['transactions'][entry]
        y = y.replace(",","")
        y = y.replace("'","")
        y = y.replace("[","")
        y = y.replace("]","")
        y = y.split()
        if "pickup" in y:
            pickup_list.append(1)
        else:
            pickup_list.append(0)
        if "delivery" in y:
            delivery_list.append(1)
        else:
            delivery_list.append(0)
        if "restaurant_reservation" in y:
            reserve_list.append(1)
        else:
            reserve_list.append(0)
    final['Pickup Available'] = pd.Series(pickup_list)
    final['Delivery Available'] = pd.Series(delivery_list)
    final["Reservation Available"] = pd.Series(reserve_list)
    my_prices = []
    for entry in range(len(final['price'])):
        if final['price'][entry]=="$":
            my_prices.append("Low")
        elif final['price'][entry]=="$$":
            my_prices.append("Medium")
        elif final['price'][entry]=="$$$":
            my_prices.append("High")
        elif final['price'][entry]=="$$$$":
            my_prices.append("Very High")
        else:
            my_prices.append(np.nan)
    final['Price Level'] = pd.Series(my_prices)
    keep = ['id','is_closed',
    'review_count',
    'rating',
    'Restaurant Category',
    'Pickup Available',
    'Delivery Available',
    "Reservation Available",
    'Price Level']
    final = final[final.columns[final.columns.isin(keep)]]
    final = final.dropna(how = 'any')
    final['is_closed'] = np.where(final['is_closed']==False,0,1)
    return final

In [202]:
frisco = pd.read_csv("YelpData_Frisco_v2.csv")
frisco2 = pd.read_csv("YelpData_Frisco.csv")
big_df = clean_yelp(frisco,frisco2)

In [203]:
big_df

,id,is_closed,review_count,rating,Restaurant Category,Pickup Available,Delivery Available,Reservation Available,Price Level
0,9VqZZiaiyGH_6smpvUKaQQ,0,127,4.5,breakfast_brunch,0,0,0,Medium
1,dPaJzxW0foPByEXbPFkh_Q,0,141,4.0,thai,1,1,0,Medium
3,yawhLqszfIp9q1aGNdLoig,0,23,4.5,tacos,1,1,0,Medium
6,amB9cpkO5890MXGctomQ9A,0,178,3.5,italian,1,1,0,Medium
10,o_g-vMKxSFZRUPicjN5s_Q,0,38,1.5,icecream,0,0,0,Low
...,...,...,...,...,...,...,...,...,...
890,Ns_3CA-0JfKKzXOAVYWOhQ,0,131,4.0,bars,0,0,0,Medium
891,2pM19ps82tA6GhsR7Wav-g,0,438,2.5,venues,0,1,0,Medium
892,azu29xbMwwXSggNxGK8NTA,0,109,3.5,hotdogs,1,1,0,Low
893,ouCXOe0T3gLxC0gxLnaMsA,0,97,3.0,salad,1,1,0,Low


In [204]:

list(big_df.columns)

['id',
 'is_closed',
 'review_count',
 'rating',
 'Restaurant Category',
 'Pickup Available',
 'Delivery Available',
 'Reservation Available',
 'Price Level']

In [205]:
big_df.to_csv("All Available Frisco Data.csv")

In [206]:
big_df.to_excel("All Available Frisco Data.xlsx")

In [207]:
big_df['is_closed'].unique()

array([0])